Using method from paper: Predicting the milk yield curve of dairy cows in the subsequent lactation

Dataset: Merged_Sorted_Data_Herd_Daily PS: cloud has made changes to this file

### data preprocessing:

In [3]:
import pandas as pd

# Load the Excel file
file_path = 'C:/Users/13593/Desktop/dsp/Data Analysis/Merged_Sorted_Data_Herd_Daily.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')
print(data.head())

,Herd Name,Date,Lac Avg Days,Weight,Rumination Minutes,Total feed,Average cell count,Day production,Expected Daily Yield,Fat indication,Fat/Protein Ratio,Protein indication,Concentrate / 100 kg Milk,Number of milkings,Total Amount of Milk Produced,Amount of Milk Separated
0,Main Herd,2016-04-29,226,710.70,375,NaN,100.0,28.5,30.4,4.49,1.22,3.68,NaN,3.1,4105.6,135.6
1,Main Herd,2016-04-30,227,715.19,379,1683.0,107.0,27.4,30.1,4.38,1.22,3.58,44.71,3.2,3952.1,146.7
2,Main Herd,2016-05-01,228,707.96,437,1682.0,113.0,24.5,29.7,4.42,1.25,3.54,43.14,3.0,3534.7,137.4
3,Main Herd,2016-05-02,229,714.28,409,1825.0,156.0,24.9,29.3,4.77,1.38,3.47,41.60,2.8,3584.9,125.5
4,Main Herd,2016-05-03,228,722.51,373,1544.0,131.0,28.2,29.1,4.46,1.34,3.32,39.77,2.8,4087.3,165.7


Missing values in the data:

In [4]:
print(data.isnull().sum())

Herd Name                          0
Date                               0
Lac Avg Days                       0
Weight                             0
Rumination Minutes                 0
Total feed                         1
Average cell count               459
Day production                     0
Expected Daily Yield               0
Fat indication                     0
Fat/Protein Ratio                  0
Protein indication                 0
Concentrate / 100 kg Milk          2
Number of milkings                 0
Total Amount of Milk Produced      0
Amount of Milk Separated           0
dtype: int64


Fill in the missing values with the median

In [5]:
data_filled = data.fillna(data.median(numeric_only=True))
print(data_filled.isnull().sum())

Herd Name                        0
Date                             0
Lac Avg Days                     0
Weight                           0
Rumination Minutes               0
Total feed                       0
Average cell count               0
Day production                   0
Expected Daily Yield             0
Fat indication                   0
Fat/Protein Ratio                0
Protein indication               0
Concentrate / 100 kg Milk        0
Number of milkings               0
Total Amount of Milk Produced    0
Amount of Milk Separated         0
dtype: int64


Change cloumn name

In [6]:
data_filled = data_filled.rename(columns=lambda x: x.replace(' ', '_'))
print(data_filled.head())

,Herd_Name,Date,Lac_Avg_Days,Weight,Rumination_Minutes,Total_feed,Average_cell_count,Day_production,Expected_Daily_Yield,Fat_indication,Fat/Protein_Ratio,Protein_indication,Concentrate_/_100_kg_Milk,Number_of_milkings,Total_Amount_of_Milk_Produced,Amount_of_Milk_Separated
0,Main Herd,2016-04-29,226,710.70,375,1186.0,100.0,28.5,30.4,4.49,1.22,3.68,31.40,3.1,4105.6,135.6
1,Main Herd,2016-04-30,227,715.19,379,1683.0,107.0,27.4,30.1,4.38,1.22,3.58,44.71,3.2,3952.1,146.7
2,Main Herd,2016-05-01,228,707.96,437,1682.0,113.0,24.5,29.7,4.42,1.25,3.54,43.14,3.0,3534.7,137.4
3,Main Herd,2016-05-02,229,714.28,409,1825.0,156.0,24.9,29.3,4.77,1.38,3.47,41.60,2.8,3584.9,125.5
4,Main Herd,2016-05-03,228,722.51,373,1544.0,131.0,28.2,29.1,4.46,1.34,3.32,39.77,2.8,4087.3,165.7


### SAE 预测缺失值

In [7]:
# 将数据转换为适合 LSTM 输入的格式。

from sklearn.model_selection import train_test_split
import numpy as np

# 选择数值型特征列
numeric_cols = data_filled.select_dtypes(include=['float64', 'int64']).columns
X = data_filled[numeric_cols].values  # 将数据转化为数组

# 为了构建SAE模型，首先需要将数据重构为3D格式 (样本数, 时间步数, 特征数)
# 假设每10天为一个时间序列段，可以根据实际情况调整
timesteps = 10
n_features = X.shape[1]

# 将数据转换为LSTM输入格式
X_sequence = []
for i in range(len(X) - timesteps):
    X_sequence.append(X[i:i+timesteps])

X_sequence = np.array(X_sequence)  # 转换为3D数组 (样本数, 时间步长, 特征数)

# 将数据集划分为训练集和测试集
X_train, X_test = train_test_split(X_sequence, test_size=0.2, random_state=42)

In [8]:
# 构建序列自编码器模型

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense

# 输入层
input_seq = Input(shape=(timesteps, n_features))

# 编码器部分
encoded = LSTM(128, activation='relu')(input_seq)  # 将序列编码为128维
encoded = RepeatVector(timesteps)(encoded)  # 将编码的向量重复，以进行解码

# 解码器部分
decoded = LSTM(128, return_sequences=True, activation='relu')(encoded)
decoded = TimeDistributed(Dense(n_features))(decoded)  # 输出重构的时间序列

# 构建自编码器模型
autoencoder = Model(inputs=input_seq, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# 打印模型结构
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 10, 14)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │          73,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector (RepeatVector)         │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 10, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 10, 14)              │           1,806 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 206,606 (807.05 KB)

 Trainable params: 206,606 (807.05 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# 训练自编码器模型
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))


Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 15s 66ms/step - loss: 893473.1250 - val_loss: 164858.2969
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 134203.3594 - val_loss: 68473.3594
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 62705.1875 - val_loss: 29669.2031
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 30028.7070 - val_loss: 65318.2734
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 54189.5977 - val_loss: 19748.4375
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - loss: 18581.8652 - val_loss: 14041.2168
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - loss: 14712.2021 - val_loss: 12103.7773
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 12281.3936 - val_loss: 12649.6582
Epoch 9/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - loss: 11919.9902 - val_loss: 9520.0254
Epoch 10/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 10514.0840 - val_loss: 8828.0869
Epoch 11/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - loss: 9197.71

In [10]:
# 使用训练好的自编码器预测缺失值
predicted_sequences = autoencoder.predict(X_test)

# 将预测结果应用到缺失的数据上
X_missing_filled = np.where(np.isnan(X_test), predicted_sequences, X_test)

# X_missing_filled 现在是填补了缺失值的完整时间序列数据


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


### Z-score standardization

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 检查列的类型，保留数值型列
numeric_cols = data_filled.select_dtypes(include=['float64', 'int64']).columns

# 提取数值型特征集
X = data_filled[numeric_cols].drop(columns=['Day_production'])  # 确保 Day_production 不在特征集中
y = data_filled['Day_production']  # 响应变量

# 使用 train_test_split 函数分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 对数据进行标准化处理
scaler = StandardScaler()

# 只在训练集上进行标准化拟合，并对训练集和测试集进行转换
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 结果转换为DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# 显示训练集的一些数据
print("Scaled Training Data:")
print(X_train_scaled_df.head())

print("\nTraining target variable:")
print(y_train.head())


TensorFlow version: 2.17.0
